# ➡️ Part 4: Multi-Step Workflows with SequentialAgent
## Content Creation Studio Workshop

Welcome back! We've built specialist agents (Part 2) and an orchestrator (Part 3). 
Now we tackle a new challenge: what if a task requires **multiple ordered steps**?

You can't write before researching. You can't format before writing. Tasks with dependencies need execution in a specific order. Today we learn about **SequentialAgent** - a workflow agent that executes sub-agents in sequence, automatically passing data between them.

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- Part 2: Giving Your Agent Custom Tools ✅
- Part 3: Building Agent Teams with Agent-as-a-Tool ✅
- **Part 4: Multi-Step Workflows with SequentialAgent** (You are here) 🎯
- Part 5: Building Iterative Workflows with LoopAgent
- Part 6: Efficient Workflows with ParallelAgent
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 4, we'll introduce these **new ADK concepts**:

1. **SequentialAgent** - Workflow agent that runs sub-agents in a specific order
2. **output_key Parameter** - Stores agent results in session state for next agent
3. **State Passing Between Agents** - Automatic data flow through workflows
4. **Variable Interpolation** - Using `{{variable}}` syntax to reference state

Each concept will be clearly marked with 🆕 when first introduced!

---

## 💡 What is a Sequential Workflow?

### 🆕 NEW CONCEPT: SequentialAgent (Workflow Agent)

> **What is SequentialAgent?**  
> A SequentialAgent is a workflow agent that executes its sub-agents in a specific order. It's designed for processes where the order of operations matters - each agent's output becomes input for the next.
>
> **Key Characteristics**:
> - Executes sub-agents one after another
> - Automatically passes state between agents
> - Useful for tasks with dependencies
> - Parameter: `sub_agents` - list of agents to run sequentially
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

### Today's Workflow:
```
Topic Research → Content Writing → Social Formatting
    ↓                ↓                  ↓
  blog_topic     blog_content     social_posts
```

Each arrow represents automatic state passing - we'll learn how this works!

## ⚙️ 1. Setup: Install Libraries

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication

In [ ]:
import os
from getpass import getpass

api_key = getpass('Enter your Google API Key: ')
os.environ['GOOGLE_API_KEY'] = api_key
print('✅ API Key configured successfully!')

## 🔬 3. Agent 1: Topic Researcher

### 🆕 NEW CONCEPT: output_key Parameter

> **What is output_key?**  
> The `output_key` parameter tells ADK to store an agent's final response in the session state under a specific variable name. This makes the output available to subsequent agents in the workflow.
>
> **How it works**:
> ```python
> agent1 = Agent(..., output_key="blog_topic")
> # Agent 1's response is stored in session.state["blog_topic"]
> ```
>
> **Benefits**:
> - Automatic state management
> - No manual data passing needed
> - Clear data flow through workflows
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

Our first agent will research topics and store the result in `blog_topic`:

**Role:** Find trending topics  
**Tool:** Google Search  
**Output:** Single best topic title  
**Key:** Sets `output_key="blog_topic"`

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

topic_research_agent = Agent(
    name="topic_research_agent",
    model="gemini-2.5-flash",
    description="Researches trending blog topics",
    instruction="""
    You are a content strategist. Find compelling blog topics.
    
    Process:
    1. Search for trending topics in the niche
    2. Select the SINGLE BEST topic
    3. Output ONLY the title
    
    Example: "10 Zero-Waste Swaps to Transform Your Kitchen"
    """,
    tools=[google_search],
    output_key="blog_topic"  # 🔑 Stores result in session.state["blog_topic"]
)

print(f"🧞 Agent '{topic_research_agent.name}' created!")

## ✍️ 4. Agent 2: Content Writer

### 🆕 NEW CONCEPT: Variable Interpolation

> **What is Variable Interpolation?**  
> ADK uses `{{variable_name}}` syntax in agent instructions to reference values from the session state. When the agent runs, ADK automatically replaces these placeholders with the actual values.
>
> **Syntax**:
> ```python
> instruction="Write about: {{blog_topic}}"
> # At runtime, {{blog_topic}} is replaced with actual value
> ```
>
> **Rules**:
> - Double curly braces: `{{variable}}`
> - Variable must exist in session state
> - Previous agents set these via `output_key`
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

Our second agent will write a blog post using the topic from Agent 1:

**Role:** Write blog post  
**Input:** Uses `{{blog_topic}}` from Agent 1  
**Output:** Complete blog post  
**Key:** Sets `output_key="blog_content"`

In [ ]:
content_writer_agent = Agent(
    name="content_writer_agent",
    model="gemini-2.5-flash",
    description="Writes engaging blog posts",
    instruction="""
    You are a blog writer. Write about: {{blog_topic}}
    
    Requirements:
    - 400-600 words
    - Engaging intro
    - 3-4 sections with H2 headings
    - Clear conclusion with CTA
    - Conversational tone
    
    Output ONLY the blog post in markdown.
    """,
    tools=[],
    output_key="blog_content"  # 🔑 Stores result in session.state["blog_content"]
)

print(f"🧞 Agent '{content_writer_agent.name}' created!")

## 📱 5. Agent 3: Social Media Formatter

Our third agent will create social posts using the blog content from Agent 2:

**Role:** Create social posts  
**Input:** Uses `{{blog_content}}` from Agent 2  
**Output:** Multi-platform social posts

In [ ]:
social_formatter_agent = Agent(
    name="social_formatter_agent",
    model="gemini-2.5-flash",
    description="Creates social media posts",
    instruction="""
    Create social posts from: {{blog_content}}
    
    Create THREE posts:
    
    1. **Twitter/X** (280 chars)
       - Hook + hashtags + CTA
    
    2. **LinkedIn** (150-200 words)
       - Professional tone
       - Key insights
       - Hashtags
    
    3. **Instagram** (150 words)
       - Engaging + emojis
       - 8-10 hashtags
       - Strong CTA
    
    Format with clear headers for each platform.
    """,
    tools=[]
    # Note: No output_key - this is the final agent
)

print(f"🧞 Agent '{social_formatter_agent.name}' created!")

## 🔗 6. Build the Sequential Workflow

### 🆕 NEW CONCEPT: State Passing Between Agents

> **How Does State Passing Work?**  
> In a SequentialAgent workflow, data flows automatically between agents:
>
> **The Flow**:
> 1. Agent 1 runs → stores output via `output_key="var1"`
> 2. Agent 2 reads `{{var1}}` from state → stores output via `output_key="var2"`
> 3. Agent 3 reads `{{var2}}` from state → produces final output
>
> **ADK Handles**:
> - Storing outputs in session state
> - Interpolating variables in instructions
> - Passing state through the workflow
> - No manual data management needed!
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

Now we chain all three agents using **SequentialAgent**:

```python
SequentialAgent(
    sub_agents=[agent1, agent2, agent3]
)
```

The ADK automatically:
1. Runs agent1 → stores output in `blog_topic`
2. Passes `blog_topic` to agent2 → stores output in `blog_content`
3. Passes `blog_content` to agent3 → final output

### Data Flow Diagram:
```
topic_research_agent (output_key="blog_topic")
         ↓ (blog_topic stored in state)
content_writer_agent (reads {{blog_topic}}, output_key="blog_content")
         ↓ (blog_content stored in state)
social_formatter_agent (reads {{blog_content}})
         ↓
    Final Output
```

In [ ]:
from google.adk.agents import SequentialAgent

content_creation_workflow = SequentialAgent(
    name="content_creation_workflow",
    sub_agents=[
        topic_research_agent,      # Step 1: Research
        content_writer_agent,      # Step 2: Write
        social_formatter_agent     # Step 3: Format
    ],
    description="Research → Write → Format workflow"
)

print("✅ Sequential workflow created!")
print("\n🔄 Execution Flow:")
print("   1. Research trending topics → blog_topic")
print("   2. Write blog post using {{blog_topic}} → blog_content")
print("   3. Format social posts using {{blog_content}} → final output")

## 🚀 7. Build Execution Engine

Our standard helper function for running queries with proper session management.

In [ ]:
from IPython.display import display, Markdown
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

session_service = InMemorySessionService()
user_id = "adk_content_creator_001"

async def run_agent_query(agent, query, session, user_id):
    print(f"\n🚀 Running: '{agent.name}'...")
    
    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    
    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"Error: {e}"
    
    print("\n" + "-"*50)
    display(Markdown(final_response))
    print("-"*50)
    
    return final_response

print("✅ Execution engine ready!")

## ✨ 8. Run the Workflow!

Watch the sequential execution:
1. 🔬 **Research** trending topics → stores `blog_topic`
2. ✍️ **Write** blog post using `{{blog_topic}}` → stores `blog_content`
3. 📱 **Format** social posts using `{{blog_content}}` → final output

All state passing happens automatically!

In [ ]:
async def run_workflow():
    session = await session_service.create_session(
        app_name=content_creation_workflow.name,
        user_id=user_id
    )
    
    query = "Create content for sustainable living and zero-waste lifestyle blog"
    print(f"👤 User: {query}\n")
    
    await run_agent_query(content_creation_workflow, query, session, user_id)

await run_workflow()

## 🎉 Recap: What We've Learned

This was a powerful step in building complex workflows:

### Core Concepts Introduced:

1. **🆕 SequentialAgent** - Workflow agent that runs sub-agents in order  
   📖 [Workflow Agents](https://google.github.io/adk-docs/agents/)

2. **🆕 output_key Parameter** - Stores agent results in session state  
   Usage: `Agent(..., output_key="variable_name")`

3. **🆕 Variable Interpolation** - References state using `{{variable}}`  
   Usage: `instruction="Write about: {{blog_topic}}"`

4. **🆕 State Passing Between Agents** - Automatic data flow  
   Agent N's output → Agent N+1's input

### Key Takeaways:

- **Order Matters**: SequentialAgent enforces execution sequence
- **Automatic State**: No manual data passing needed
- **Clean Interface**: `output_key` + `{{variable}}` = simple state flow
- **Composable**: Agents become pipeline stages

### Architecture Pattern:
```
Topic Research (output_key="blog_topic")
       ↓ (stores in state)
Content Writer (uses {{blog_topic}}, output_key="blog_content")
       ↓ (stores in state)
Social Formatter (uses {{blog_content}})
       ↓
Final Multi-Platform Content
```

---

## 🚀 What's Next?

We can now create ordered workflows where data flows automatically between steps! But what if we need **iterative refinement**? What if content needs to be reviewed and improved until it meets quality standards?

In **Part 5**, we'll learn about **LoopAgent** - a workflow agent that can critique and improve its own work through iterative cycles until quality thresholds are met.

### Preview of Part 5 Concepts:
- 🆕 LoopAgent (Workflow Agent)
- 🆕 ToolContext
- 🆕 tool_context.actions.escalate
- 🆕 Critique-Refine Pattern

See you in Part 5! 🚀